# Le changement climatique peut il être observé dans les Alpes?

## Consignes

Ce TP peut être réalisé sur [MyDocker]. XX TODO ADD
Vous devrez rendre ce notebook avec les fonctions/ votre script python ainsi qu'un compte-rendu de 8 pages maximum dans lequel vous répondrez aux questions.
**Le compte-rendu au format PDF et le notebook** sont à rendre par email avant le prochain cours soit **lundi 17 janvier 23h59**. J'ouvrirai mes mails au réveil, et à partir de là, -2 points par heure de retard.

## Introduction

En science des données vous devrez préparer les données, les analyser (statistiquement) et produire des figures pertinentes dans l'objectif de répondre à différentes questions.

Dans ce TP, on se demande si le changement climatique est visible dans les Alpes et nous mettrons en lien les observations avec les analyses des flux hydrométriques (= le force du courant dans les rivières) dans cette région. 

Pour ce faire nous allons commencer par étudier l'évolution de la météo au cours des dernières décennies. Météo France, l'organisme national de météorologie en France, a déposé des données climatologiques par département avec de nombreux paramètres disponibles sur le site [data.gouv](https://www.data.gouv.fr/fr/datasets/donnees-climatologiques-de-base-quotidiennes/).

Dans un second temps, nous étudierons l'évolution des débits de l'Arve (une rivière) en lien avec le changement climatique et la fonte des glaciers aux alentours du Mont-Blanc (un massif *relativement* connu des Alpes). Le papier a été publié dans Scientific Reports en 2020 et est disponible [ici](https://doi.org/10.1038/s41598-020-67379-7).

## Chargement des librairies

Voici quelques librairies dont vous aurez très probablement besoin, n'hésitez pas a en ajouter d'autres !

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

%matplotlib inline
!pip install folium
import folium


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


## Optimisation du code

Pensez à optimiser votre code pour améliorer l'efficacité, en particulier lorsqu'il s'agit de manipuler de grandes quantités de données. Utilisez des fonctions pour que vous puissiez refaire la même analyse avec un autre département.

# PARTIE 1: Données climatiques de Météo France

## Chargement du jeu de données 

Commencez par récupérer sur le site des données publiques francaises, les données quotidiennes de météo entre 1950 et 2022 en Haute-Savoie où se situe le Mont-Blanc (département n°74) sur le site [data.gouv](https://www.data.gouv.fr/fr/datasets/donnees-climatologiques-de-base-quotidiennes/).

On ne s'intéresse pour le moment qu'aux données de température et de précipitation.

Quand vous êtes prête ou prêt, chargez la table d'intéret dans la variable `meteo`.

*Indication :* Pour trouver quel fichier télécharger, lisez la documentation!

In [ ]:
meteo = pd.read_csv("Q_74_previous-1950-2023_RR-T-Vent.csv", sep =';')
#print(meteo.head())
display(meteo.head(10).style.set_table_styles(
    [{'selector': 'th', 'props': [('font-size', '12pt'), ('text-align', 'center')]},
     {'selector': 'td', 'props': [('font-size', '10pt'), ('text-align', 'center')]}]
))

## Exploration des données

Décrivez la table. Par exempl,e répondez aux questions suivantes:

- Quelle sont les dimensions de `meteo` ?
- Combien y a-t-il de postes météorologiques ? 
    
Vos réponses sont à rédiger dans le compte-rendu pas dans le notebook, ici je n'évaluerai que les fonctions!

In [ ]:
dim = meteo.shape
print(dim)

nbr_poste = meteo['NUM_POSTE'].nunique()
print(nbr_poste)

In [ ]:
print(meteo.columns)


In [ ]:
meteo.describe()

**Gestion des valeurs manquantes et filtration**

- Quelles colonnes allez vous sélectionner pour notre étude ? Pour rappel, on va étudier l'évolution de la température et des précipitations depuis les années 1950 dans les Alpes.
*Indication:* La localisation des postes pourra être utile.
- Créer la table `meteo_cleaned` avec les colonnes d'interet et sans données manquantes.
- Faites une première analyse sur les informations statistiques de base sur les données (moyenne, médiane, écart-type, etc.).


In [ ]:
meteo = meteo.rename(columns={
    "AAAAMMJJ": "Date", "RR": "Précipitations", "TNTXM": "TNTXM",
    "LAT": "Latitude", "LON": "Longitude", "ALTI": "Altitude", 
    "NOM_USUEL": "Nom_poste", "NUM_POSTE": "NUM_POSTE"
})


In [ ]:
colonnes_interet = ["Date", "Précipitations", "TNTXM", 
                    "Latitude", "Longitude", "Altitude", 
                    "Nom_poste", "NUM_POSTE"]

# Sélection des colonnes
meteo = meteo[colonnes_interet]

# Remplacement des valeurs manquantes
#meteo["Précipitations"] = meteo["Précipitations"].fillna(0)
#meteo["Température_moyenne"] = meteo["Température_moyenne"].fillna(meteo["Température_moyenne"].mean())
#meteo["Température_maximale"] = meteo["Température_maximale"].fillna(meteo["Température_maximale"].mean())
#meteo["Température_minimale"] = meteo["Température_minimale"].fillna(meteo["Température_minimale"].mean())

# Nettoyage des données
meteo.dropna(subset=['TNTXM'], inplace=True)

meteo_cleaned = meteo.copy()


# Affichage des premières lignes
print(meteo_cleaned.head())


In [ ]:
print(meteo_cleaned.columns)


In [ ]:
stats = meteo_cleaned[["Précipitations", "TNTXM"]].describe()
print("\nAnalyse statistique :")
print(stats)

In [ ]:
meteo_cleaned.columns

In [ ]:
meteo_cleaned.shape

In [ ]:
meteo_cleaned.describe()

- Combien de stations restent dans notre étude ? Où se situent-elles ?

In [ ]:
stations_restantes = meteo_cleaned['NUM_POSTE'].nunique()
print(f"Nombre de stations restantes dans l'étude : {stations_restantes}")


In [ ]:
stations_localisation = meteo_cleaned[['NUM_POSTE', 'Nom_poste', 'Latitude', 'Longitude', 'Altitude']].drop_duplicates()
print(stations_localisation)


In [ ]:

plt.figure(figsize=(10, 6))
plt.scatter(stations_localisation['Longitude'], stations_localisation['Latitude'], c='blue', alpha=0.7)
plt.title('Localisation des stations restantes')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid()
plt.show()


In [ ]:

center_lat = stations_localisation['Latitude'].mean()
center_lon = stations_localisation['Longitude'].mean()

map_stations = folium.Map(location=[center_lat, center_lon], zoom_start=6)

for _, row in stations_localisation.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Station: {row['Nom_poste']} ({row['NUM_POSTE']})",
    ).add_to(map_stations)

map_stations


## Analyse des données

### Tendances annuelles

Quelles sont les tendances annuelles dans les données météorologiques depuis 1950 ? La température moyenne a-t-elle changée ? Est-ce qu'il y a plus ou moins de précipitations ?

Faites une analyse (calcul de moyenne, tests de regressions etc) pour répondre à la question. Ci-dessous voici les principales étapes à effectuer:
- Transformez la colonne date pour que vous puissiez l'exploiter facilement
- Calculez les températures et précipitations moyennes annuelles
- Faites une régression linéaire pour estimer l'évolution de ces valeurs. Utilisez scikit-learn comme appris en L1 ISD.
- Evaluez la performance de vos modèles de prédiction. Pour cela, vous pourrez par exemple utiliser les métriques disponibles dans `sklearn.metrics`.
- La régression est-elle pertinente pour la température ? Pour les précipitations ?
- Auriez-vous un autre modèle plus pertinent qu'une régression linéaire à proposer ? N'hésitez pas à l'**ajouter**.
- Crééz également un ou plusieurs graphiques pour représenter les variations des paramètres météorologiques au fil du temps.

Si besoin vous pouvez charger de nouvelles librairies.

In [ ]:
# Faire une copie explicite du DataFrame
meteo_cleaned = meteo_cleaned.copy()

# Conversion de la colonne Date en format datetime
meteo_cleaned["Date"] = pd.to_datetime(meteo_cleaned["Date"], format='%Y%m%d')

# Extraction de l'année
meteo_cleaned["Année"] = meteo_cleaned["Date"].dt.year


In [ ]:
print(meteo_cleaned["Année"].unique())


In [ ]:
# Calcul des moyennes annuelles
moyennes_annuelles = meteo_cleaned.groupby("Année")[["TNTXM", "Précipitations"]].mean().reset_index()
print(moyennes_annuelles.head())


In [ ]:
print(moyennes_annuelles.columns)


In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(moyennes_annuelles['Année'], moyennes_annuelles['TNTXM'])
plt.xlabel('Année')
plt.ylabel('Température')
plt.title('La Température Annuelle Moyenne ')
plt.show()

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(moyennes_annuelles['Année'], moyennes_annuelles['Précipitations'])
plt.xlabel('Année')
plt.ylabel('Précipitation')
plt.title('Précipitation moyenne annuelle ')
plt.show()

In [ ]:
year_avg = pd.pivot_table(meteo_cleaned, values='TNTXM', index='Année', aggfunc='mean')
year_avg['10 Years MA'] = year_avg['TNTXM'].rolling(10).mean() ## j'ai appliqué un moving avearge de 10 
year_avg[['TNTXM','10 Years MA']].plot(figsize=(20,6))
plt.title('Variation de la temperature')
plt.xlabel('Année')
plt.ylabel('Temperature')
plt.xticks([x for x in range(1950,2022,3)])
plt.show()

In [ ]:
#Gestion des valeurs abérrantes
num_atts = num_atts = ['TNTXM', 'Précipitations']

fig, axes = plt.subplots(1, 2, figsize=(20, 10)) 

for i, att in enumerate(num_atts):
    sns.boxplot(ax=axes[i], x=att, data=meteo_cleaned)

plt.show()

In [ ]:
iqr = meteo_cleaned['TNTXM'].quantile(0.75) - meteo_cleaned['TNTXM'].quantile(0.25)
l1 = meteo_cleaned['TNTXM'].quantile(0.25) - 1.5*iqr
l2 = meteo_cleaned['TNTXM'].quantile(0.75) + 1.5*iqr


data_1 = meteo_cleaned[np.logical_and(meteo_cleaned['TNTXM'] < l2, meteo_cleaned['TNTXM'] > l1)]
data_1.describe(include = 'all')
data_1 = data_1.copy()
data_1.head()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Modèle pour la température moyenne
X_temp = moyennes_annuelles["Année"].values.reshape(-1, 1)
y_temp = moyennes_annuelles["TNTXM"].values

# Modèle pour les précipitations
X_precip = moyennes_annuelles["Année"].values.reshape(-1, 1)
y_precip = moyennes_annuelles["Précipitations"].values

# Régression linéaire pour la température
model_temp = LinearRegression()
model_temp.fit(X_temp, y_temp)
y_temp_pred = model_temp.predict(X_temp)

# Régression linéaire pour les précipitations
model_precip = LinearRegression()
model_precip.fit(X_precip, y_precip)
y_precip_pred = model_precip.predict(X_precip)

# Évaluation des modèles
mse_temp = mean_squared_error(y_temp, y_temp_pred)
r2_temp = r2_score(y_temp, y_temp_pred)

mse_precip = mean_squared_error(y_precip, y_precip_pred)
r2_precip = r2_score(y_precip, y_precip_pred)

print("Température :")
print(f"MSE : {mse_temp}, R² : {r2_temp}")

print("\nPrécipitations :")
print(f"MSE : {mse_precip}, R² : {r2_precip}")


In [ ]:
import matplotlib.pyplot as plt

# Visualisation des tendances
plt.figure(figsize=(12, 6))

# Température moyenne
plt.subplot(1, 2, 1)
plt.scatter(moyennes_annuelles["Année"], moyennes_annuelles["TNTXM"], color='blue', label='Données')
plt.plot(moyennes_annuelles["Année"], y_temp_pred, color='red', label='Régression linéaire')
plt.xlabel("Année")
plt.ylabel("Température moyenne")
plt.title("Tendance de la température moyenne")
plt.legend()

# Précipitations
plt.subplot(1, 2, 2)
plt.scatter(moyennes_annuelles["Année"], moyennes_annuelles["Précipitations"], color='green', label='Données')
plt.plot(moyennes_annuelles["Année"], y_precip_pred, color='orange', label='Régression linéaire')
plt.xlabel("Année")
plt.ylabel("Précipitations moyennes")
plt.title("Tendance des précipitations moyennes")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Transformation polynomiale
degree = 3  # Choisis le degré du polynôme
poly = PolynomialFeatures(degree=degree)
X_poly = poly.fit_transform(X_temp)  # Transformation de X_temp pour inclure les termes polynomiaux

# Modèle de régression linéaire avec les nouvelles données
poly_model = LinearRegression()
poly_model.fit(X_poly, y_temp)

# Prédictions
y_poly_pred = poly_model.predict(X_poly)

# Évaluation du modèle
mse_poly = mean_squared_error(y_temp, y_poly_pred)
r2_poly = r2_score(y_temp, y_poly_pred)

# Affichage des résultats
print(f"Régression polynomiale (degré {degree}) :")
print(f"MSE : {mse_poly}")
print(f"R² : {r2_poly}")

# Visualisation des données et des prédictions
plt.scatter(X_temp, y_temp, color='blue', label='Données réelles')
plt.plot(X_temp, y_poly_pred, color='red', label='Prédictions (polynomiale)')
plt.xlabel('Année')
plt.ylabel('Température moyenne')
plt.title('Régression polynomiale (degré 3)')
plt.legend()
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Séparer les données en ensembles d'entraînement et de test
X_temp_train, X_temp_test, y_temp_train, y_temp_test = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42
)
X_precip_train, X_precip_test, y_precip_train, y_precip_test = train_test_split(
    X_precip, y_precip, test_size=0.2, random_state=42
)

# Modèle : Forêt Aléatoire pour la Température
rf_temp = RandomForestRegressor(n_estimators=100, random_state=42)
rf_temp.fit(X_temp_train, y_temp_train)
y_temp_rf_pred = rf_temp.predict(X_temp_test)

# Modèle : Forêt Aléatoire pour les Précipitations
rf_precip = RandomForestRegressor(n_estimators=100, random_state=42)
rf_precip.fit(X_precip_train, y_precip_train)
y_precip_rf_pred = rf_precip.predict(X_precip_test)

# Évaluation des modèles Forêt Aléatoire
mse_rf_temp = mean_squared_error(y_temp_test, y_temp_rf_pred)
r2_rf_temp = r2_score(y_temp_test, y_temp_rf_pred)

mse_rf_precip = mean_squared_error(y_precip_test, y_precip_rf_pred)
r2_rf_precip = r2_score(y_precip_test, y_precip_rf_pred)

print("\nForêt Aléatoire - Température :")
print(f"MSE : {mse_rf_temp:.4f}, R² : {r2_rf_temp:.4f}")

print("\nForêt Aléatoire - Précipitations :")
print(f"MSE : {mse_rf_precip:.4f}, R² : {r2_rf_precip:.4f}")


### Prévisions

Quelle température fera-t-il en 2100 selon votre modèle ?


In [ ]:
from sklearn.linear_model import LinearRegression

# Entraîner un modèle de régression linéaire
lin_reg = LinearRegression()
lin_reg.fit(X_temp_train, y_temp_train)

# Prédire la température pour l'année 2100
X_2100 = [[2100]]  # L'année seule comme caractéristique
y_2100_pred = lin_reg.predict(X_2100)

print(f"Température prédite en 2100 : {y_2100_pred[0]:.2f}°C")


### Variabilité saisonnière 

Analysez la variabilité saisonnière des données. 

- Manipulez les données pour les regrouper mois par mois. Y a-t-il des tendances ? Correspondent-elles à vos connaissances ?
- N'oubliez pas de visualisez les données
- Enfin si vous êtes à l'aise, étudiez en plus les épisodes de fortes chaleurs (températures > 28°C). Est-ce qu'il y en a plus souvent plus récemment?



In [ ]:
meteo_cleaned.columns

In [ ]:
# je vais créer une table pivot pour tracer les températures mensuelles au fil des années
meteo_cleaned.set_index('Date', inplace=True)
meteo_cleaned['Mois'] = meteo_cleaned.index.month
meteo_cleaned['Année'] = meteo_cleaned.index.year

In [ ]:
pivot = pd.pivot_table(meteo_cleaned, values='TNTXM', index='Mois', columns='Année', aggfunc='mean')
pivot.plot(figsize=(20,6))
plt.title('Variation de la temperature par mois')
plt.xlabel('Mois')
plt.ylabel('Temperatures')
plt.xticks([x for x in range(1,13)])
plt.legend().remove()
plt.show()

In [ ]:
monthly_seasonality = pivot.mean(axis=1)
monthly_seasonality.plot(figsize=(20,6))
plt.title('Variation de la temperature par mois')
plt.xlabel('Mois')
plt.ylabel('Temperatures')
plt.xticks([x for x in range(1,13)])
plt.show()

In [ ]:
meteo_cleaned.columns

In [ ]:
#corrélation
plt.figure(figsize=(8, 6))
plt.scatter(meteo_cleaned['TNTXM'], meteo_cleaned['Précipitations'])
plt.title('Graphique de Dispersion entre TNTXM et RR')
plt.xlabel('TNTXM')
plt.ylabel('RR')
plt.show()

In [ ]:
meteo_cleaned = meteo_cleaned.drop(['Année' , 'NUM_POSTE' , 'Nom_poste' ], axis = 1 ) 
correlation_matrix = meteo_cleaned.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Matrice de corrélation')
plt.show()

# PARTIE 2: Evolution des débits de l'Arve


Dans cet [article](https://doi.org/10.1038/s41598-020-67379-7), les autrices et les auteurs examinent l'impact du changement climatique et de la perte de masse des glaciers sur l'hydrologie du massif du Mont-Blanc, en particulier sur le bassin versant de la rivière Arve, qui est alimentée par une série de glaciers dans la région. Ils ont utilisé des projections climatiques (scénarios RCP4.5 et RCP8.5) et des simulations de dynamique des glaciers (historiques et futures) combinées a un modèle hydrologique pour étudier l'évolution du débit des rivières à l'échelle du 21e siècle (vus en cours).

Vous avez représenté ci-dessous la zone d'étude de l'article avec en panneau (a), la localisation du bassin versant de Sallanches dans les Alpes françaises et la carte des bassins étudiés et en panneau (b), le massif du Mont-Blanc vu de Téléphérique de la Flégère (point noir avec angle de vue sur le plan).
![Zone étudiée: Fig 1 du papier](Fig1.png)


Dans leur article, ils commencent par étudier l'évolution des températures et des précipitations à Sallances. Ils montrent qu'il y a une nette augmentation des températures estivales et hivernales dans la région du Mont-Blanc et des modificaions de précipitations. Cela correspond à la figure 2 représentée ci-dessous. **Avez-vous obtenus les mêmes résultats avec votre analyse de la partie 1?**


![Fig 2 du papier](Fig2.png)



Dans cette partie, nous allons reproduire la figure n°3 du papier qui illustre l'évolution des débits saisonniers de la rivière Arve en fonction des scénarios climatiques. La figure présente des courbes pour les débits moyens (en hiver et en été) simulées sous les scénarios RCP4.5 et RCP8.5, à partir de différents modèles climatiques pour la région du Mont-Blanc.

## Chargement des données

Dans le dossier `Donnees_Debits/`, vous disposez de données simulées historiques ainsi que de projections futures des débits cumulés (et de leur écart-type) de la rivière Arve (pour chaque scénario climatique). Ces données sont organisées en fonction des saisons (hiver, été ou moyenne annuelle) et des différents scénarios climatiques (historiques, RCP4.5, RCP8.5). 
Vous disposez également des séries temporelles de débits observés (`Debits_obs_Sal_sans_2902`) pour la saison d'hiver et d'été, qui servent de référence pour la comparaison des simulations.

**Regardez un peu les données. Quelles sont les dimensions des données historiques et des données de modélisation ?**

*Indications: Pour rappel, les deux scénarios RCP font partie des trajectoires d'émissions de gaz à effet de serre utilisées pour projeter les futurs changements climatiques. Pour chacun d'eux, plusieurs modèles climatiques régionaux ont été utilisés pour simuler l'évolution du climat et ensuite du débit de la rivière. Les différents modèles régionaux sont des modèles climatiques spécifiques, souvent *downscalés* (affinés à une échelle régionale) pour mieux simuler les conditions climatiques locales du Mont Blanc. Le processus de downscaling permet d'obtenir des projections climatiques à une résolution spatiale plus fine que celle des modèles climatiques globaux.* 


In [ ]:
import os

# Dossier contenant les fichiers
data_folder = "Donnees_Debits/"

# Fonction 1 : Chargement des fichiers selon le scénario et la saison
def load_files(scenario, season):
    cum_file = f"Donnees_Debits/Q_cum_{scenario}_{season}_multimodeles_OK"
    sd_file = f"Donnees_Debits/Q_sd_{scenario}_{season}_multimodeles_OK"
    cum_data = pd.read_csv(cum_file, sep=',', parse_dates=['Date'])
    sd_data = pd.read_csv(sd_file, sep=',', parse_dates=['Date'])

    return cum_data, sd_data

## Analyse et création de fonctions

**Etant donné une saison, faites une fonction qui pour chaque scenario (historique, ou RCP), calcule la moyenne et l'écart-type moyen du débit pour chaque année. Commencez par charger les données et transformer la date pour pouvoir extraire l'année. Dans un second temps, chargez les données observées et extrayez la valeur pour la saison considérée. Puis, reproduisez les graphiques montrant l'évolution du débit moyen pour les saisons d'hiver et d'été, sous les deux scénarios RCP4.5 et RCP8.5, comparés aux données historiques et aux données observées.**

Est-ce clair pour vous, quelle est la différence entre les données observées et historiques ?


**Discutez comment le changement climatique impacte les ressources en eau dans les régions montagneuses.**



In [ ]:
# Fonction 2 : Visualisation selon le scénario et la saison
def visualize_discharge(scenario, season):
    # Charger les fichiers
    cum_data, sd_data = load_files(scenario, season)

    # Calcul des moyennes annuelles
    cum_data_annual = cum_data.groupby('Date').mean().reset_index()
    sd_data_annual = sd_data.groupby('Date').mean().reset_index()

    # Calculer la moyenne des colonnes de débit
    mean_discharge = cum_data_annual.iloc[:, 1:].mean(axis=1)
    sd_discharge = sd_data_annual.iloc[:, 1:].mean(axis=1)

    # Visualisation avec intervalle SD
    plt.figure(figsize=(12, 6))
    plt.plot(cum_data_annual['Date'], mean_discharge, label=f'{scenario.upper()} - Mean', linestyle='-', marker='o', color='blue')
    plt.fill_between(cum_data_annual['Date'], 
                     mean_discharge - 2 * sd_discharge, 
                     mean_discharge + 2 * sd_discharge, 
                     color='blue', alpha=0.2, label=f'{scenario.upper()} - SD Interval')
    plt.title(f'Évolution des débits moyens et SD ({season.capitalize()}) - {scenario.upper()}')
    plt.xlabel('Année')
    plt.ylabel('Débit (m³/s)')
    plt.legend()
    plt.grid()
    plt.show()

# Visualisation groupée par saison
def visualize_by_season(season):
    """
    Visualise les débits pour tous les scénarios dans une saison donnée.
    :param season: Saison ("hiver", "ete", "annuele").
    """
    plt.figure(figsize=(14, 8))
    for scenario in ["hist", "RCP4-5", "RCP8-5"]:
        cum_data, sd_data = load_files(scenario, season)

        
        # Calcul des moyennes annuelles
        cum_data_annual = cum_data.groupby('Date').mean().reset_index()
        sd_data_annual = sd_data.groupby('Date').mean().reset_index()

        # Calculer la moyenne des colonnes de débit
        mean_discharge = cum_data_annual.iloc[:, 1:].mean(axis=1)
        sd_discharge = sd_data_annual.iloc[:, 1:].mean(axis=1)

        # Ajouter les courbes avec intervalle SD
        plt.plot(cum_data_annual['Date'], mean_discharge, label=f'{scenario.upper()} - Mean ({season})', linestyle='-', marker='o')
        plt.fill_between(cum_data_annual['Date'], 
                         mean_discharge - 15 * sd_discharge, 
                         mean_discharge + 15 * sd_discharge, 
                         alpha=0.2, label=f'{scenario.upper()} - SD Interval ({season})')

    plt.title(f'Évolution des débits moyens et SD - {season.capitalize()}')
    plt.xlabel('Année')
    plt.ylabel('Débit (m³/s)')
    plt.legend()
    plt.grid()
    plt.show()

# Exemple d'utilisation
visualize_by_season("hiver")
visualize_by_season("ete")
visualize_by_season("annee")